In [21]:
import sys
sys.path.append("../../utils")
sys.path.append("../../")

import pdb

import requests
import pandas as pd
import re

import signal
import time

import pymongo

from keys import *

In [2]:
def has_key(key, resp, pageid):
    return bool(resp['query']["pages"][pageid].get(key))

def matches(lst1, lst2):
    return [x for x in lst1 if x in lst2]
    
    
def get_article_info(title):
    params = {
        "action": "query",
        "format": "json",

        "titles": title,

        "prop": "extracts|info|links|linkshere|categories",
        
        # extracts
        "exintro": True,
        "explaintext": True,
        "exsectionformat": "plain",
        
        # links
        "pllimit": "max",
        "plnamespace": 0,
        
        # linkshere
        "lhlimit": "max",
        "lhnamespace": 0,
        "lhshow": "!redirect",
        
        # categories
        "cllimit": "max",
    }

    extract = []
    links = []
    linkshere = []
    categories = []

    def query_info(title, params):

        resp = requests.get(
            url="https://en.wikipedia.org/w/api.php",
            params=params).json()

        pageid = list(resp["query"]['pages'].keys())[0]
        
        if has_key("extract", resp, pageid):
            extract.append(resp['query']["pages"][pageid]['extract'])
        
        if has_key("links", resp, pageid):
            for link in resp['query']["pages"][pageid]["links"]:
                links.append(link["title"])
        
        if has_key("linkshere", resp, pageid):
            for lh in resp['query']["pages"][pageid]["linkshere"]:
                linkshere.append(lh["title"])
        
        if has_key("categories", resp, pageid):
            for cat in resp['query']["pages"][pageid]["categories"]:
                if not bool(re.findall(r"(articles)|(uses)|(commons)", cat["title"], re.I)):
                    categories.append(cat["title"])
        
        if resp.get('continue'):
            params.update(resp.get("continue"))
            query_info(title, params)

    query_info(title, params)
    
    return extract, links, linkshere, categories

In [26]:
class WikiAPI:

    def __init__(self, df, collection="article_links"):

        self.df = df
        
        # setup timeout function 
        def handle_alarm(signum, frame):
            raise RuntimeError
            
        signal.signal(signal.SIGALRM, handle_alarm)
        
        
    def get_article_info(self, title):
        try:
            signal.alarm(5) # allow up to 5 seconds for each api call
            extract, links, linkshere, categories = get_article_info(title)
            formatted_info = {"title": title, "extract": extract, "links": links, "linkshere": linkshere, "categories": categories}
            signal.alarm(0)
            return formatted_info
        except:
            return None

    def unique_articles(self):
        parent = self.df.parent_title
        child = self.df.child_title        
        return list(parent.append(child).unique())
        
    def get_api_info(self):
        
        articles = self.unique_articles()        
        
        for article_title in articles:            
            
            article_info = self.get_article_info(article_title)
            
            if article_info:
                

        
    def seen_to_csv(self):
        for title, relation in self.seen.items():
            self.seen[title] = list(relation)
            
        seen_df = pd.DataFrame(self.seen).T.reset_index()
        seen_df.columns = ["name", "relation"]
        seen_df.to_csv("article_title_relationships.csv", index=False)

In [12]:
related_articles = pd.read_csv("../../data/raw/05_expanded_labaled_removed_comparisons.zip")

In [27]:
# test_set = related_articles[0:2].copy()
api = WikiAPI(related_articles[0:4])

api.test_signal()
# api.get_api_info()

RuntimeError: 